## Utility functions, CS4243, Amir


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.linalg import hadamard

%matplotlib inline

In [2]:
# function to compute the entropy of an image. 
# nimg is the input image and N is the number of graylevels, default is 256
# nimg is supposed to be a graylevel image.
#
def am_entropy(nimg , N=256):
    M = nimg.shape
    ssz = M[0] * M[1]
    hist,bins = np.histogram(nimg.flatten(),N,[0,N])
    hist = hist / ssz
    ent = -np.sum( hist * np.log2(hist+0.000001))
    return ent


In [3]:
# function to compute the image power. input could be graylevel or color. 
#
def am_power(a):
    dim1 = a.shape
    
    if len(dim1)==2:
        sz = dim1[0] * dim1[1] 
    elif len(dim1)==3:
        sz = dim1[0] * dim1[1] * dim1[2]        
    pa = np.sum( a ** 2.0) / sz    
    
    return pa

In [4]:
# test ....
print( am_power( np.ones([8,8])) , am_power( np.ones([81,81]))*30 )

1.0 30.0


In [5]:
# function to compute the image energy. input could be graylevel or color. 

am_energy = lambda a : np.sum(np.multiply(a,a)) 


In [6]:
# test ....
print( am_energy( np.ones([8,8])) , am_energy( np.ones([8,8]))*30 )

64.0 1920.0


In [7]:
# function to computer tha valid part of an image after convolution
# a is image, and N is the size of filter convolved previosly with a
#
def am_valid_part(a,N=3):
    cff = int(N/2)
    M = a.shape
    return a[cff:M[0]-cff, cff:M[1]-cff, :]

## make a matrix ready to be shown as an image, ... , 

In [8]:
def ready_2_show(a, level=255):
    a = ( a - np.min(a) ) / (np.max(a) - np.min(a)) 
    a = a * level
    return np.uint8(a)


## Frequency Domain Filters definition
### here, we define a few frequency domain filter that can be used to filter
### images in Fourier, Wlash/Hadamard, and Cosine domain
### filter max is 1 and mean is 0
### 

In [9]:
#
# The function generates an ideal low pass filter for frequency domain, 
# M and N are size of the filter/image, D0 is the cut_off point. 
# 
def idealLowPass(M, N, D0):
    # Initializing the filter with ones; since the filter is a complex function,
    # it has two channels, representing the real and imaginary parts:
    filter = np.ones((M, N), dtype=np.uint8)
    # normalized cut_off frequency is mapped to real index
    D0 = D0 * min(M,N) / 2
    # Scanning through each pixel and calculating the distance of each pixel
    # to the image center. If the pixel is within D0, it is changed to 0:
    for i in range(M):
        for j in range(N):
            if ( (i-M/2)**2 + (j-N/2)**2)**0.5 >= D0:
                filter[i,j]= 0
            
    return filter


In [10]:
f= idealLowPass(10,10,0.3)

In [11]:
print(f)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [12]:
f3= idealLowPass(200,300,0.3)
cv2.imshow("ideal low pass" , ready_2_show(f3) )


In [13]:
#
# The function generates an ideal high pass filter for frequency domain, 
# M and N are size of the filter/image, D0 is the cut off point. 
# 
idealHighPass = lambda M, N, D0 : 1- idealLowPass(M,N,D0)

In [14]:
f= idealHighPass(10,10,0.4)

In [15]:
print(f)

[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 0 0 1 1 1]
 [1 1 1 1 0 0 0 1 1 1]
 [1 1 1 1 0 0 0 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


In [16]:
# 
# this function generates a lowpass Gaussian filter
# image and filter size is MxN , STD of the Gaussian is D0 which is also the cut_off
# point of the filter
# 
def GaussLowPass(M, N, D0):
    filter = np.zeros((M, N))
    # normalized cut_off frequency is mapped to real index
    D0 = D0 * min(M,N) / 2
    for i in range(M):
        for j in range(N):
            d = ( (i-M/2)**2 + (j-N/2)**2 )**0.5
            filter[i,j]= np.exp(-((d/2/D0)**2) )
            
    return filter


In [17]:
f= GaussLowPass(8, 8, 0.8) 
print(f)

[[0.45783336 0.54315988 0.61368025 0.66031449 0.67663385 0.66031449
  0.61368025 0.54315988]
 [0.54315988 0.64438872 0.72805199 0.78337746 0.80273827 0.78337746
  0.72805199 0.64438872]
 [0.61368025 0.72805199 0.82257756 0.88508613 0.90696062 0.88508613
  0.82257756 0.72805199]
 [0.66031449 0.78337746 0.88508613 0.9523448  0.97588155 0.9523448
  0.88508613 0.78337746]
 [0.67663385 0.80273827 0.90696062 0.97588155 1.         0.97588155
  0.90696062 0.80273827]
 [0.66031449 0.78337746 0.88508613 0.9523448  0.97588155 0.9523448
  0.88508613 0.78337746]
 [0.61368025 0.72805199 0.82257756 0.88508613 0.90696062 0.88508613
  0.82257756 0.72805199]
 [0.54315988 0.64438872 0.72805199 0.78337746 0.80273827 0.78337746
  0.72805199 0.64438872]]


In [18]:
f4 = GaussLowPass(250, 250, 0.3)
cv2.imshow("Gaussian low pass" , ready_2_show(f4) )

In [19]:
GaussHighPass = lambda M, N, D0: 1- GaussLowPass(M, N, D0)

In [20]:
hh = GaussHighPass(250, 250, 0.85)
cv2.imshow("Gaussian high pass" , ready_2_show(hh) )

In [21]:
# 
# to generate a lowpass Butterworth filter of the frequency domain  
# filter size is MxN, cut_off point of the filter is D0, n_o is the filter order
#
def ButterworthLowPass(M, N, D0, n_o):
    #  
    filter = np.zeros((M, N))
    # normalized cut_off frequency is mapped to real index
    D0 = D0 * min(M,N) / 2
    n_o = 2 * n_o
    for i in range(M):
        for j in range(N):
            d = ( (i-M/2)**2 + (j-N/2)**2 )**0.5
            filter[i,j]= 1 / ( 1 + (d/D0)**n_o )
            
    return filter


In [22]:
f= ButterworthLowPass(10, 10, 0.5, 1) 
print(f)

[[0.11111111 0.13227513 0.1552795  0.17730496 0.19379845 0.2
  0.19379845 0.17730496 0.1552795  0.13227513]
 [0.13227513 0.16339869 0.2        0.23809524 0.2688172  0.28089888
  0.2688172  0.23809524 0.2        0.16339869]
 [0.1552795  0.2        0.25773196 0.32467532 0.38461538 0.40983607
  0.38461538 0.32467532 0.25773196 0.2       ]
 [0.17730496 0.23809524 0.32467532 0.43859649 0.55555556 0.6097561
  0.55555556 0.43859649 0.32467532 0.23809524]
 [0.19379845 0.2688172  0.38461538 0.55555556 0.75757576 0.86206897
  0.75757576 0.55555556 0.38461538 0.2688172 ]
 [0.2        0.28089888 0.40983607 0.6097561  0.86206897 1.
  0.86206897 0.6097561  0.40983607 0.28089888]
 [0.19379845 0.2688172  0.38461538 0.55555556 0.75757576 0.86206897
  0.75757576 0.55555556 0.38461538 0.2688172 ]
 [0.17730496 0.23809524 0.32467532 0.43859649 0.55555556 0.6097561
  0.55555556 0.43859649 0.32467532 0.23809524]
 [0.1552795  0.2        0.25773196 0.32467532 0.38461538 0.40983607
  0.38461538 0.32467532 0.257

In [23]:
f =ButterworthLowPass(250, 250, 0.3 , 1 )
cv2.imshow("Butterworth low pass, order=1" , ready_2_show(f) ) 
g =ButterworthLowPass(250, 250, 0.3 , 4 )
cv2.imshow("Butterworth low pass, order=4" , ready_2_show(g) )

In [24]:
ButterworthHighPass = lambda M, N, D0, n_o: 1 - ButterworthLowPass(M, N, D0, n_o)

In [25]:
# 
# to generate a lowpass Butterworth filter of the frequency domain  
# filter size is MxN, cut_off point of the filter is D0, n_o is the filter order
#
def ButterworthBandPass(M, N, D0, D1, n_o):
    #  
    filter = ButterworthLowPass(M, N,D0,n_o)
    filter = ButterworthLowPass(M, N,D1,n_o) - filter
        
    filter = filter * (1/np.max(filter)) 
    
    return filter

In [26]:
ff = ButterworthBandPass(512,512,0.05, 0.2 ,2) 
print(np.min(ff) , ' --- ', np.max(ff))
g = (np.abs(ff-0.5) <= 0.01)
g = np.uint8(g)
cv2.imshow("BP Butterworth",ready_2_show(ff))
cv2.imshow("Cut_off freqs",ready_2_show(g))


0.0  ---  1.0


In [27]:
ButterworthBandReject = lambda M, N, D0, D1, n_o: 1 - ButterworthBandPass(M, N, D0, D1, n_o)

In [28]:
ff2 = ButterworthBandReject(512,512,0.05, 0.2 ,2) 
print(np.min(ff2) , ' --- ', np.max(ff2))
gg = (np.abs(ff2-0.5) <= 0.01)
gg = np.uint8(gg)
cv2.imshow("BR Butterworth",ready_2_show(ff2))
cv2.imshow("Cut_off freqs",ready_2_show(gg))


0.0  ---  1.0


In [ ]:
def am_print_mat( a ):
    M = a.shape 
    for i in range(M[0]):
        for j in range(M[1]): 
            print('%7.3f '%(a[i][j]) ,end='')
        print(' ')


In [29]:
cv2.waitKey(0)
cv2.destroyAllWindows() 